<a href="https://colab.research.google.com/github/changwoolab/20251R0136COSE47400/blob/densenet-121/densenet_121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade torch torchvision tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [2]:
!pip install --upgrade kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2


In [3]:
from google.colab import files
uploaded = files.upload()  # select cifake-real-and-ai-generated-synthetic-images.zip
for fn in uploaded.keys():
    !unzip -q "{fn}" -d data



Saving archive.zip to archive.zip


In [4]:
pip install scikit-learn

In [5]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import classification_report

# 1) Data transforms
train_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.485, .456, .406],
                         std=[.229, .224, .225])
])
val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.485, .456, .406],
                         std=[.229, .224, .225])
])

# 2) Datasets & loaders
train_ds = datasets.ImageFolder("data/train", transform=train_tfms)
val_ds   = datasets.ImageFolder("data/test",   transform=val_tfms)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4)

# 3) Model
model = models.densenet121(pretrained=True)
# replace the classifier head: DenseNet features -> 2 classes
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=De

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 50.4MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train_epoch(loader):
    model.train()
    running_loss = 0
    correct = 0
    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc  = correct / len(loader.dataset)
    return epoch_loss, epoch_acc

def eval_epoch(loader):
    model.eval()
    running_loss = 0
    correct = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            running_loss += criterion(outputs, labels).item() * imgs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return running_loss / len(loader.dataset), correct / len(loader.dataset)

# 4) Main training loop
best_acc = 0
os.makedirs("models", exist_ok=True)
for epoch in range(1, 11):
    train_loss, train_acc = train_epoch(train_loader)
    val_loss,   val_acc   = eval_epoch(val_loader)
    print(f"Epoch {epoch:02d} | "
          f"Train: {train_loss:.3f}, {train_acc:.3%} | "
          f"Val:   {val_loss:.3f}, {val_acc:.3%}")
    # save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "models/densenet121_best.pth")
         # load the best model
    model.load_state_dict(torch.load("models/densenet121_best.pth"))
    # gather preds
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc="Evaluating"):
            imgs = imgs.to(device)
            out  = model(imgs)
            y_pred.extend(out.argmax(1).cpu().tolist())
            y_true.extend(labels.tolist())
    # print report
    print("\nClassification report:\n")
    print(classification_report(y_true, y_pred, target_names=['FAKE','REAL'], digits=4))


100%|██████████| 3125/3125 [17:29<00:00,  2.98it/s]


Epoch 01 | Train: 0.138, 94.632% | Val:   0.090, 96.690%


Evaluating: 100%|██████████| 625/625 [01:11<00:00,  8.72it/s]



Classification report:

              precision    recall  f1-score   support

        FAKE     0.9544    0.9806    0.9673     10000
        REAL     0.9801    0.9532    0.9664     10000

    accuracy                         0.9669     20000
   macro avg     0.9673    0.9669    0.9669     20000
weighted avg     0.9673    0.9669    0.9669     20000



  0%|          | 0/3125 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 3125/3125 [17:34<00:00,  2.96it/s]


...

  0%|          | 0/3125 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 3125/3125 [17:33<00:00,  2.97it/s]


Epoch 05 | Train: 0.056, 97.926% | Val:   0.068, 97.585%


Evaluating: 100%|██████████| 625/625 [01:12<00:00,  8.61it/s]



Classification report:

              precision    recall  f1-score   support

        FAKE     0.9815    0.9704    0.9759     10000
        REAL     0.9707    0.9817    0.9762     10000

    accuracy                         0.9760     20000
   macro avg     0.9761    0.9761    0.9760     20000
weighted avg     0.9761    0.9760    0.9760     20000



  0%|          | 0/3125 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
 95%|█████████▌| 2979/3125 [16:43<00:51,  2.85it/s]